In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)
from Performance import performance_evaluation
perfs = performance_evaluation()

from Performance import H2h_Regret_Evaluation
h2h_regret_eva = H2h_Regret_Evaluation()

from Data import Data_Simulator
DS_Obj = Data_Simulator()

from Oracle import Oracle_Processing
Oracle_Proc = Oracle_Processing()

from OLS import OLS_Processing
OLS_Proc = OLS_Processing()

from DDR import DDR_Processing
DDR_Proc = DDR_Processing()

from PYEPO import EPO_Processing
PYEPO_Proc = EPO_Processing()

from Data_Load_Store import Load_Store_Methods
Data_LSM = Load_Store_Methods()

### Parameters

In [2]:
grid = (5,5) # grid size
from Network import network_design
Network = network_design()
arcs,arc_index_mapping = Network._getArcs(grid)

num_test = 1000
lower = 0 # coef lower bound
upper = 1 # coef upper bound
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100

In [3]:
data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
project_directory = os.path.dirname(os.path.dirname(os.path.dirname(parent_directory)))
DataPath_Parent = project_directory + '/Data_JOC_R1/SPO_vs_OLS' + data_generation_process + f'_coef_seed={coef_seed}/{grid[0]}by{grid[1]}_Network/'
pathlib.Path(DataPath_Parent).mkdir(parents=True, exist_ok=True)
print("DataPath_parent:", DataPath_Parent)

DataPath_parent: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/SPO_vs_OLSSPO_Data_Generation_coef_seed=1/5by5_Network/


#### Impact of sample size

In [4]:
num_feat = 5 # size of feature
p = num_feat
deg = 4.0 # polynomial degree
mis = deg # model misspecification
e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term
iteration_all = np.arange(0,100)
batch_size = 20
num_epochs = 1000

num_train = 1000
DataPath = DataPath_Parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("DataPath:", DataPath)

x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = \
DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/SPO_vs_OLSSPO_Data_Generation_coef_seed=1/5by5_Network/data_size=1000_deg=4.0_e=0.5_d=40_p=5_x_dist=uniform_num_test=1000/


In [5]:
cost_Oracle_Post_all,cost_Oracle_Ante_all = Oracle_Proc.Implement_Oracle(arcs, grid,mis,bump,\
                                                            W_star_all,x_test_all,noise_test_all,\
                                                            iteration_all,num_feat,data_generation_process)

cost_OLS_Post_all,cost_OLS_Ante_all = OLS_Proc.Implement_OLS(arcs, grid,mis,bump,\
                                                    W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,\
                                                    iteration_all,num_feat,data_generation_process)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle: iter= 20 ,cost_Oracle_Ante= 626.4473818047255
Oracle: iter= 40 ,cost_Oracle_Ante= 617.7016863812473
Oracle: iter= 60 ,cost_Oracle_Ante= 554.1906641376092
Oracle: iter= 80 ,cost_Oracle_Ante= 588.7090244734214
OLS: iter= 20 ,cost_OLS_Ante= 633.2784570095941
OLS: iter= 40 ,cost_OLS_Ante= 630.5775247374638
OLS: iter= 60 ,cost_OLS_Ante= 563.4125539439547
OLS: iter= 80 ,cost_OLS_Ante= 603.5799879345628


In [6]:
batch_size = 20
num_epochs = 1000
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()
method_names = ["spo+"]
cost_EPO_Post_all,cost_EPO_Ante_all = PYEPO_Proc.Implement_EPO(DataPath,iteration_all,batch_size,num_epochs,method_names,\
                                            W_star_all,bump,x_train_all,c_train_all,x_test_all,noise_test_all,\
                                            arcs,grid,perfs,num_feat,mis,data_generation_process)

['spo+'] : iter= 20 ,cost_EPO_Ante= 642.33930882729


Process ForkPoolWorker-2:
Traceback (most recent call last):
KeyboardInterrupt



Interrupt request received


  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/lib/python3.9/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook/Decision_Driven_Regularization/.venv/

GurobiError: Unable to retrieve attribute 'x'